In [1]:
import cripser

import tcripser
from persim import plot_diagrams

import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

import warnings
warnings.filterwarnings("ignore")

# Functions:

In [4]:
def interactive_plots(im):
  max_im = np.max(im)
  margin = max_im/15
  fig = go.FigureWidget() ; colormap = 'gray' #'cool'
  @interact(parameter=(max_im/25,max_im+margin,(max_im)/100))
  def update(parameter=0):
      with fig.batch_update(): 
        try:
          im_sublevel = im.copy() ; im_sublevel[im_sublevel>=parameter] = parameter
          holes = im.copy()       ; holes[holes<=parameter] = parameter
          _,ax = plt.subplots(1,4,figsize=(25,5))
          divider = make_axes_locatable(ax[0])
          cax = divider.append_axes('right', size='5%', pad=0.05)
          ims = ax[0].imshow(im,          cmap=colormap) ; ax[0].set_title('Original image')
          _.colorbar(ims, cax=cax, orientation='vertical')
          ax[1].imshow(holes, cmap=colormap) ; ax[1].set_title('Sub-level set of the image')

          pd = tcripser.computePH(im_sublevel,maxdim=2)
          pd0 = pd[pd[:,0]==0][:,1:3] ; pd1 = pd[pd[:,0]==1][:,1:3]
          dgms = [pd0] ; dgms.append(pd1)
          dgms[0][:,1][dgms[0][:,1]>=parameter]=parameter 
          dgms[1][:,1][dgms[1][:,1]>=parameter]=parameter 

          y_ticks_H0 = np.linspace(margin, max_im/2-margin, len(dgms[0][:,0]))
          y_ticks_H1 = np.linspace(max_im/2, max_im-margin, len(dgms[1][:,0]))
          ax[2].hlines(y=y_ticks_H0, xmin=dgms[0][:,0], xmax=dgms[0][:,1], linestyle='-', colors='g', label=r'$H_0$')
          ax[2].hlines(y=y_ticks_H1, xmin=dgms[1][:,0], xmax=dgms[1][:,1], linestyle='--', colors='r', label=r'$H_1$')
          ax[2].set_xlim(0,max_im+margin); ax[2].set_ylim(0,max_im); ax[2].set_yticks([]); ax[2].set_title('Persistence barcode') ; ax[2].set_xlabel('Parameter')
          ax[2].legend()

          plot_diagrams(dgms, ax=ax[3], xy_range=[0-margin, max_im+margin*1.1, 0-margin, max_im+margin*1.1]) ; ax[3].set_title('Persistence diagram')
        except: pass

def circle_creator(points = 1000, coef = 30, radius = 10, j_coeff = 10, Noise = .05, plot=False, jj=1, circles_num=7):
  from sklearn.datasets import make_circles
  xy = make_circles(points, noise=Noise, shuffle=False)[0][:int(points/2)]

  stacked_xy = xy.copy()
  for j,i in enumerate(np.linspace(0, 2*np.pi, circles_num)):
    new_points = points*(j+1)*j_coeff
    xy =     make_circles(new_points, noise=Noise, shuffle=False)[0][:int(new_points/2)]
    new_xy = make_circles(new_points, noise=Noise, shuffle=False)[0][:int(new_points/2)]
    if jj == 1:
      new_xy[:,0] = xy[:,0]*radius + np.sin(i)*coef
      new_xy[:,1] = xy[:,1]*radius + np.cos(i)*coef
    else:
      new_xy[:,0] = xy[:,0]*radius*j/jj + np.sin(i)*coef
      new_xy[:,1] = xy[:,1]*radius*j/jj + np.cos(i)*coef
      
    stacked_xy = np.vstack((stacked_xy, new_xy))
  x , y = stacked_xy[:,0][points:] , stacked_xy[:,1][points:]
  if plot==False: return x,y
  else: plt.plot(x,y,'.');

def joint_pdf(points, Min=False, Max=False, resolution=64, gaussian_filter_sigma=0):
    from scipy.ndimage import gaussian_filter

    if Min==False: Min=np.min(points)
    if Max==False: Max=np.max(points)
    xx = points[:,0] ; yy = points[:,1]
    
    land_scape = np.vstack((xx,yy)).T
    M = resolution ; joint = np.zeros((M,M)) ; ls = land_scape
    dy = dx = (np.max(ls)-Min)/(M-1)
    if dx==0: dx=1
    if dy==0: dy=1
    for i in range(len(ls)):
        kx = int((ls[i,0]-Min)/dx)
        ky = int((ls[i,1]-Min)/dy)
        joint[ky][kx] += 1
    #joint /= (np.sum(joint)*dx*dy)
    return gaussian_filter(joint, gaussian_filter_sigma)

In [5]:
im0 = np.array([[0,0,0,0,0],
                [0,1,0,2,0],
                [0,0,0,0,0],
                [0,4,0,3,0],
                [0,0,0,0,0]], dtype=float)

interactive_plots(im0)

interactive(children=(FloatSlider(value=0.16, description='parameter', max=4.266666666666667, min=0.16, step=0…

In [6]:
im1 = np.array([[0,0,0,0,0,0,0,0,0],
                [0,1,1,1,0,2,2,2,0],
                [0,1,0,1,0,2,0,2,0],
                [0,1,1,1,0,2,2,2,0],
                [0,0,0,0,0,0,0,0,0],
                [0,4,4,4,0,3,3,3,0],
                [0,4,0,4,0,3,0,3,0],
                [0,4,4,4,0,3,3,3,0],
                [0,0,0,0,0,0,0,0,0]], dtype=float)

interactive_plots(im1)

interactive(children=(FloatSlider(value=0.16, description='parameter', max=4.266666666666667, min=0.16, step=0…

In [7]:
x,y = circle_creator(jj=1)#, circles_num=20, Noise=1)
im2 = joint_pdf(np.vstack((x,y)).T, resolution=64)

# plt.figure(figsize=(12,5))
# plt.subplot(121) ; plt.plot(x,y,',') ; plt.grid() ; plt.title('Point cloud')
# plt.subplot(122) ; plt.imshow(im2, cmap='gray') ; plt.colorbar() ; plt.title('Image') ; 

interactive_plots(im2)

interactive(children=(FloatSlider(value=27.96, description='parameter', max=745.6, min=27.96, step=6.99), Outp…